# Célula de inicialização

#### Aqui todas as funções e libs necessárias para o código funcionar são iniciadas. Digno de nota: Antes de saber que era possível utilizar "%%time" para mensurar o tempo de execução de uma célula, montei uma função para retornar esse valor toda vez que um código fosse executado. Apesar de ser algo simples, é extremamente útil.

In [8]:
%%time

import pandas as pd
import shutil as sh
import numpy as np
import time as t
import xml.etree.ElementTree as et
from xml.dom import minidom
import os
import re


def exec_time(i,f):
    tempo = f - i
    minutos = tempo // 60
    segundos = tempo % 60
    if tempo >= 60:
        return print(f"Tempo de execução: {minutos:.0f}m{segundos:.1f}s")
    elif tempo >= 1 and tempo < 60:
        return print(f"Tempo de execução: {tempo:.0f}s")
    elif 0 <= tempo < 1:
        return print(f"Tempo de execução: {tempo:.3f}s")
    else:
        return("Tem alguma coisa errada aí, bicho.")
    
def Filler(matriz,pivo):
    c = 0
    for lista in matriz:
        lista.insert(0, pivo[c])
        c = c + 1
    
def Stores(local,indice):
    cod = []
    for item in local[indice]:
        sliced = item.split("Loja: ")[::-1]
        codigo = sliced[0].strip()
        codigo = re.sub("<br />", '', codigo)
        if len(codigo) == 19:
            cod.append("Amazon")
        elif codigo[0:5] == "Lojas":
            cod.append("Americanas")
        elif codigo[0:4] == "Subm":
            cod.append("Americanas")
        elif codigo[0:4] == "Shop":
            cod.append("Americanas")
        elif codigo[0:1] == "LU":
            cod.append("Magalu")
        elif len(codigo) == 15:
            cod.append("Carrefour")
        elif len(codigo) == 14:
            cod.append("Shopee")
        elif len(codigo) == 11:
            cod.append("Centauro")
        elif len(codigo) == 10:
            cod.append("Dafiti")
        elif len(codigo) == 9:
            cod.append("Fristyle")
        elif len(codigo) == 8:
            cod.append("Netshoes")
        elif "X" in codigo and codigo[0:1] != "Lo":
            cod.append("Amazon FBA OS")
        else:
            cod.append("Informação Ausente")
    local.pop(indice)
    local.insert(indice,cod)
    return local

def move(o1,o2,d1,d2):
    sh.move(o1,d1)
    sh.move(o2,d2)
    
C1 = t.time()    
C2 = t.time()
exec_time(C1,C2)

Tempo de execução: 0.000s
Wall time: 0 ns


## Célula de extração

#### É nesta célula que os cabeçalhos serão definidos em listas, matrizes serão montadas para armazenar os dados extraídos diretamente das notas fiscais eletrônicas - como todas as NFe aqui são notas "limpas" (sem notas de cancelamento ou evento no "bolo"), não houve necessidade de implementar estruturas condicionais na parte de leitura e parseamento.

In [10]:
faturamento = ["Operação","Data","Nome","Cidade","UF",
           "Loja","Número da NF","Itens comprados","Valor da Venda","Valor do Frete","Desconto","Total"]
produtos = ["Data","Loja","Código","Produto","Quantidade comprada","Valor do item","Número da NF"]

path = os.chdir(# Inserir diretório)

lista1 = [[] for item in range(len(faturamento))]
lista2 = [[] for item in range(len(produtos))]

for item in os.listdir(path):
    if item.endswith(".xml"):
        xml = open(item)
        nfe = minidom.parse(xml)
        if item.endswith("-can.xml"):
            continue
        
        # COMANDOS - DATAFRAME DE FATURAMENTO
        op = nfe.getElementsByTagName('natOp')
        data = nfe.getElementsByTagName('dhEmi')
        nome = nfe.getElementsByTagName('xNome')
        cidade = nfe.getElementsByTagName('xMun')
        UF = nfe.getElementsByTagName('UF')
        loja = nfe.getElementsByTagName('infCpl')
        NF = nfe.getElementsByTagName('cNF')
        itens = nfe.getElementsByTagName('det')
        prod = nfe.getElementsByTagName('vProd')
        frete = nfe.getElementsByTagName('vFrete')
        desc = nfe.getElementsByTagName('vDesc')
        total = nfe.getElementsByTagName('vNF')
        
        # COMANDOS - DATAFRAME DE PRODUTOS
        cod_produto = nfe.getElementsByTagName('cProd')
        data_prod = nfe.getElementsByTagName("dhEmi")
        d_prod = data * len(cod_produto)
        NF = nfe.getElementsByTagName("cNF")
        NF_prod = NF * len(cod_produto)
        loja_prod = nfe.getElementsByTagName('infCpl')
        loja_prod_2 = loja_prod * len(cod_produto)
        nome_produto = nfe.getElementsByTagName('xProd')
        qtde_comprada = nfe.getElementsByTagName('qCom')
        valor_item = nfe.getElementsByTagName('vUnCom')

    
        # LISTAS- DATAFRAME DE FATURAMENTO
        lista1[0].append(op[0].firstChild.data)
        lista1[1].append(data[0].firstChild.data)
        lista1[2].append(nome[0].firstChild.data)
        lista1[3].append(cidade[1].firstChild.data)
        lista1[4].append(UF[1].firstChild.data)
        lista1[5].append(loja[0].firstChild.data)
        lista1[6].append(NF[0].firstChild.data)
        lista1[7].append(len(itens))
        lista1[8].append(prod[-1].firstChild.data)
        lista1[9].append(frete[-1].firstChild.data)
        lista1[10].append(desc[-1].firstChild.data)
        lista1[11].append(total[0].firstChild.data)
        
        # LISTAS - DATAFRAME DE PRODUTOS
        for d in d_prod:
            lista2[0].append(d.firstChild.data)
        for p in loja_prod_2:
            lista2[1].append(p.firstChild.data)
        for i in cod_produto:
            lista2[2].append(i.firstChild.data)
        for j in nome_produto:
            lista2[3].append(j.firstChild.data)
        for k in qtde_comprada:
            lista2[4].append(k.firstChild.data)
        for l in valor_item:
            lista2[5].append(l.firstChild.data)
        for n in NF_prod:
            lista2[6].append(n.firstChild.data)

## Célula de transformação - entram listas, saem dataframes inteiros

In [ ]:
Stores(lista1,5)
Filler(lista1,faturamento)
Stores(lista2,1)
Filler(lista2,produtos)

nfe_dict1 = {x[0]: x[1:] for x in lista1}
df = pd.DataFrame(nfe_dict1)
nfe_dict2 = {x[0]: x[1:] for x in lista2}
df2 = pd.DataFrame(nfe_dict2)

for item in [df,df2]:
    item['Data'] = pd.to_datetime(item.Data)
    item['Data'] = item['Data'].dt.strftime('%d/%m/%Y')
    item['Número da NF'] = item['Número da NF'].astype(str)

df = df.sort_values(by=['Data'])
df2  = df2.sort_values(by=['Data'])

for col in df.columns[6:]:
    df[col] = pd.to_numeric(df[col], errors='coerce')
for col in df2.columns[4:]:
    df2[col] = pd.to_numeric(df2[col], errors='coerce')
    

df.to_excel("FATURAMENTO.xlsx")
df2.to_excel("PRODUTOS.xlsx")

## Célula de transferência de arquivos

In [11]:
origem1 = # Inserir diretório de origem + nome do arquivo
origem2 = # Inserir diretório de origem + nome do arquivo
dest1 = # Inserir diretório de destino + nome do arquivo
dest2 = # Inserir diretório de destino + nome do arquivo

move(origem1,origem2,dest1,dest2)

## Nota final:
##### Posteriormente, pretendo escrever sobre este código em um artigo do Medium - afinal, ele é a minha primeira grande tarefa dentro do estágio que estou. Motivações, tempo de desenvolvimento, as etapas, etc, tudo será devidamente escrito. Também pretendo realizar updates - apesar de ser poderoso, ele pode e será otimizado para tarefas cada vez mais complexas.